In [12]:
!pip install --upgrade git+https://github.com/taghizad3h/DeCLUTR.git

  Cloning https://github.com/taghizad3h/DeCLUTR.git to /tmp/pip-req-build-z6l2liz_
  Running command git clone -q https://github.com/taghizad3h/DeCLUTR.git /tmp/pip-req-build-z6l2liz_
  Resolved https://github.com/taghizad3h/DeCLUTR.git to commit 200e0739d6bf63baa333f3e5b1e8aefc68a7decd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for declutr: filename=declutr-0.1.0rc1-py3-none-any.whl size=41843 sha256=39a7f6039a8ea82c14c3ca5d93bd4a168926ad13be97deb94ace08bb109022c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-mqhixm0y/wheels/d7/cd/00/ad17ef4982f002307ff387de689d4eb9453687fe64daedf12b
Successfully built declutr


In [ ]:
!pip install --upgrade transformers

In [6]:
# transformer_model = "aubmindlab/bert-base-arabertv02"
transformer_model = 'arabert-v02'
# transformer_model = "HooshvareLab/distilbert-fa-zwnj-base"

In [18]:
declutr_small_config = f"""
// This should be a registered name in the Transformers library (see https://huggingface.co/models) 
// OR a path on disk to a serialized transformer model.
local transformer_model = "{transformer_model}";

// This will be used to set the max/min # of tokens in the positive and negative examples.
local max_length = 64;
local min_length = 32;

{{
    "vocabulary": {{
        "type": "empty"
    }},
    "dataset_reader": {{
        "type": "declutr",
        "num_anchors": 2,
        "num_positives": 2,
        "max_span_len": max_length,
        "min_span_len": min_length,
        "manual_multiprocess_sharding": true,
        "tokenizer": {{
            "type": "pretrained_transformer",
            "model_name": transformer_model,
            // Account for special tokens (e.g. CLS and SEP), otherwise a cryptic error is thrown.
            "max_length": max_length - 2,
        }},
        "token_indexers": {{
            "tokens": {{
                "type": "pretrained_transformer",
                "model_name": transformer_model,
            }},
        }},
    }}, 
    "train_data_path": "ar_books.txt",
    "model": {{
        "type": "declutr",
        "text_field_embedder": {{
            "type": "mlm",
            "token_embedders": {{
                "tokens": {{
                    "type": "pretrained_transformer_mlm",
                    "model_name": transformer_model,
                    "masked_language_modeling": true
                }},
            }},
        }},
        "loss": {{
            "type": "nt_xent",
            "temperature": 0.05,
        }},
    }},
    "data_loader": {{
        "batch_size": 16,
        "num_workers": 8,
        "drop_last": true,
        "max_instances_in_memory": 16 * 100,
        "batches_per_epoch": 1024,
    }},
    "trainer": {{
        // Set use_amp to true to use automatic mixed-precision during training (if your GPU supports it)
        "use_amp": true,
        "optimizer": {{
            "type": "huggingface_adamw",
            "lr": 5e-5,
            "eps": 1e-06,
            "correct_bias": false,
            "weight_decay": 0.0,
            "parameter_groups": [
                // Apply weight decay to pre-trained params, excluding LayerNorm params and biases
                // See: https://github.com/huggingface/transformers/blob/2184f87003c18ad8a172ecab9a821626522cf8e7/examples/run_ner.py#L105
                // Regex: https://regex101.com/r/ZUyDgR/3/tests
                [["(?=.*transformer_model)(?=.*\\\.+)(?!.*(LayerNorm|bias)).*$"], {{"weight_decay": 0.1}}],
            ],
        }},
        "num_epochs": 1,
        "checkpointer": {{
            // A value of null or -1 will save the weights of the model at the end of every epoch
            "keep_most_recent_by_count": null,
        }},
        "grad_norm": 1.0,
        "learning_rate_scheduler": {{
            "type": "slanted_triangular",
        }},
    }},
}}
"""
# print(declutr_small_config)
with open('declutr_small.jsonnet', 'w') as f:
    f.write(declutr_small_config)

In [21]:
overrides = (
    f"{{'train_data_path': 'ar_books.txt', "
    # lower the batch size to be able to train on Colab GPUs
    f"'data_loader.batch_size': 16, "
    # training examples / batch size. Not required, but gives us a more informative progress bar during training
    f"'data_loader.batches_per_epoch': 1024}}"
)

In [17]:
import os
os.environ["ALLENNLP_LOG_LEVEL"] = "ERROR" 
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [16]:
!allennlp train "declutr_small.jsonnet" \
    --serialization-dir "output" \
    --include-package "declutr" \
    -f
#     --overrides "$overrides" \

2021-08-24 11:35:03.678808: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
setting eval mode
evalmode is set to False
Some weights of the model checkpoint at arabert-v02 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassi

In [ ]:
!wget -nc https://raw.githubusercontent.com/JohnGiorgi/DeCLUTR/master/scripts/save_pretrained_hf.py

--2021-06-03 14:09:16--  https://raw.githubusercontent.com/JohnGiorgi/DeCLUTR/master/scripts/save_pretrained_hf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1878 (1.8K) [text/plain]
Saving to: ‘save_pretrained_hf.py’

save_pretrained_hf. 100%[===================>]   1.83K  --.-KB/s    in 0s      

2021-06-03 14:09:16 (43.1 MB/s) - ‘save_pretrained_hf.py’ saved [1878/1878]



In [ ]:
!python save_pretrained_hf.py /content/output/model.tar.gz declutr_transformer

2021-06-03 14:09:21.850964: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
💾 🤗 Transformers compatible model saved to: declutr_transformer. See https://huggingface.co/transformers/model_sharing.html for instructions on hosting the model with 🤗 Transformers.


In [ ]:
!nvidia-smi

Wed Aug  4 11:19:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
class A:
    def __init__(self, a):
        self.a = a
        
class B(A):
    def __init__(self, a, b):
        super().__init__(a)
        self.b = b
        
        print(self.a)
        

b = B('1', '2')
        

1
